# Eksperyment i porównanie modeli regresji liniowej, regularyzacji Ridge (grzbietowa), Lasso oraz SVM
W celu przeprowadzenia poprawnie eksperymentu należy wykonać walidację krzyżową, czyli dokonać kilkukrotnego podziału na zbiór uczący i treningowy. Zbadane zostaną cztery modele w imprementacji `sklearn`:
regresja liniowa, regularyzacja Lasso, regularyzacja grzbietowa (Ridge) oraz SVM dla przypadku regresji czyli Support Vector Regression. Początkowo wykonano implementacje wszystkich modeli z parametrami domyślnymi.


In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import KFold
import copy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from Goal import goal_function

In [2]:
df = pd.read_excel('data.xlsx')
np.random.seed(1410)

clfs = {
    'LinReg   ': LinearRegression(),
    'Ridge    ': Ridge(),
    'Lasso    ': Lasso(),
    'SVR      ': SVR()
}

# Walidacja krzyżowa
times_cross_validation = 3
kf = KFold(n_splits=times_cross_validation, shuffle=True, random_state=1410)
scaler = MinMaxScaler()

X = df.drop(columns=['PE'])
y = df['PE']
result_basic = np.zeros((times_cross_validation, len(clfs)))
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for j, clf_key in enumerate(clfs):
        clf = copy.deepcopy(clfs[clf_key])
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        result_basic[i, j] = mean_absolute_error(y_pred, y_test)

for i, clf_key in enumerate(clfs):
    print(f'MAE dla {clf_key} wynosi {round(result_basic[:,i].mean(),3)}')

MAE dla LinReg    wynosi 3.628
MAE dla Ridge     wynosi 3.632
MAE dla Lasso     wynosi 5.616
MAE dla SVR       wynosi 3.186


Następnie powtórzono eksperyment z wyznaczonymi wcześniej parametrami:

In [3]:
ridge_param = {'alpha': -1.4950000000000534, 'solver': 'auto', 'random_state': 46}
lasso_param = {'alpha': 0.01, 'fit_intercept': True, 'max_iter': 700, 'random_state': 46}
svr_param = {'C': 1.3, 'epsilon': 0.11}

clfs = {
    'LinReg   ': LinearRegression(),
    'Ridge    ': Ridge(**ridge_param),
    'Lasso    ': Lasso(**lasso_param),
    'SVR      ': SVR(**svr_param)
}

np.random.seed(1410)
times_cross_validation = 3
kf = KFold(n_splits=times_cross_validation, shuffle=True, random_state=1410)
scaler = MinMaxScaler()

X = df.drop(columns=['PE'])
y = df['PE']
result_tun = np.zeros((times_cross_validation, len(clfs)))
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for j, clf_key in enumerate(clfs):
        clf = copy.deepcopy(clfs[clf_key])
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        result_tun[i, j] = mean_absolute_error(y_pred, y_test)

for i, clf_key in enumerate(clfs):
    print(f'MAE dla {clf_key} wynosi {round(result_tun[:,i].mean(),3)}')

MAE dla LinReg    wynosi 3.628
MAE dla Ridge     wynosi 3.625
MAE dla Lasso     wynosi 3.629
MAE dla SVR       wynosi 3.169


Kolejnym punktem było zaszumienie danych i przetestowanie działania algorytmów. W tym celu każda wartość przemnożono raz wartośc wylosowaną z rozkładu normalnego o wartości oczekiwanej 1 i odchyleniu standardowym o wartości 0.1.

In [10]:
df = pd.read_excel('data.xlsx')
columns_name = np.array(df.columns)
noise = np.random.normal(1, 0.1, [len(columns_name), len(df)])
for i, name in enumerate(columns_name):
    df[name] = df[name]*noise[i]

In [11]:
for i, name in enumerate(columns_name):
    df[name] = df[name]*noise[i]

In [13]:
ridge_param = {'alpha': -1.4950000000000534, 'solver': 'auto', 'random_state': 46}
lasso_param = {'alpha': 0.01, 'fit_intercept': True, 'max_iter': 700, 'random_state': 46}
svr_param = {'C': 1.3, 'epsilon': 0.11}

clfs = {
    'LinReg   ': LinearRegression(),
    'Ridge    ': Ridge(**ridge_param),
    'Lasso    ': Lasso(**lasso_param),
    'SVR      ': SVR(**svr_param)
}

np.random.seed(1410)
times_cross_validation = 3
kf = KFold(n_splits=times_cross_validation, shuffle=True, random_state=1410)
scaler = MinMaxScaler()

X = df.drop(columns=['PE'])
y = df['PE']
result_noise = np.zeros((times_cross_validation, len(clfs)))
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for j, clf_key in enumerate(clfs):
        clf = copy.deepcopy(clfs[clf_key])
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        result_noise[i, j] = mean_absolute_error(y_pred, y_test)

for i, clf_key in enumerate(clfs):
    print(f'MAE dla {clf_key} wynosi {round(result_noise[:,i].mean(),3)}')

MAE dla LinReg    wynosi 71.498
MAE dla Ridge     wynosi 71.499
MAE dla Lasso     wynosi 71.499
MAE dla SVR       wynosi 71.461


In [17]:
from prettytable import PrettyTable
t = PrettyTable(['Klasyfikator', 'Parametry domyślne', 'Po strojeniu', 'Zaszumione'])
for i, clf_key in enumerate(clfs):
    if i==0:
        t.add_row([clf_key, round(result_basic[:,i].mean(), 3), '-', '-'])
    else:
        t.add_row([clf_key, round(result_basic[:,i].mean(), 3), round(result_tun[:,i].mean(), 3), 
               round(result_noise[:,i].mean(), 3)])
print(t)

+--------------+--------------------+--------------+------------+
| Klasyfikator | Parametry domyślne | Po strojeniu | Zaszumione |
+--------------+--------------------+--------------+------------+
|  LinReg      |       3.628        |      -       |     -      |
|  Ridge       |       3.632        |    3.625     |   71.499   |
|  Lasso       |       5.616        |    3.629     |   71.499   |
|  SVR         |       3.186        |    3.169     |   71.461   |
+--------------+--------------------+--------------+------------+
